In [1]:
import numpy as np
import os, sys
from pathlib import Path
import uproot

sys.path.append("/users/hep/eigen1907/Workspace/Workspace-RPC/modules")
from NanoAODTnP.Plotting.PlotEfficiency import init_figure, hist_eff_by_roll, get_region_params

In [2]:
import json
import uproot
import numpy as np
import pandas as pd
import mplhep as mh

from pathlib import Path
from typing import Optional, Union
from hist import intervals
from datetime import datetime
from datetime import timedelta

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import LogNorm
from matplotlib.patches import Rectangle

def init_figure(
    figsize: tuple = (8, 6),
    fontsize: float = 20,
    com: float = 13.6,
    label1: str = 'Work in Progress',
    label2: Optional[str] = None,
    mid_label: Optional[str] = None,
    lumi: Optional[float] = None,
    lumi_format = "{0: .1f}",
    loc: int = 2,
    xlabel: Optional[str] = None,
    ylabel: Optional[str] = None,
    xlim: Optional[tuple] = None,
    ylim: Optional[tuple] = None,
    xticks: Optional[list] = None,
    yticks: Optional[list] = None,
    log_scale: bool = False,
) -> plt.Figure:
    mh.style.use(mh.styles.CMS)
    fig, ax = plt.subplots(figsize = figsize)
    mh.cms.label(ax=ax, data=True, label=label1, loc=loc,
                 year=label2, com=com, fontsize=fontsize,
                 lumi=lumi, lumi_format=lumi_format)
    ax.set_xlabel(xlabel, fontsize=fontsize)
    ax.set_ylabel(ylabel, fontsize=fontsize)
    if mid_label is not None:
        ax.annotate(mid_label, (0.50, 1.015), #weight='bold',
                    xycoords='axes fraction', fontsize=fontsize, horizontalalignment='center')
    if xlim is not None:
        ax.set_xlim(xlim)
    if ylim is not None:    
        ax.set_ylim(ylim)
    if xticks is not None:
        ax.set_xticks(xticks)
    if yticks is not None:
        ax.set_yticks(yticks)
    if log_scale == True:
        ax.set_yscale(log)
    return fig, ax

def get_region_params(region: str) -> dict:
    facecolor_table = {'All': ['#8EFFF9', '#00AEC9'],
                       'Barrel': ['#d3f5e4', '#21bf70'],
                       'Disk1,2,3': ['#7CA1FF', '#0714FF'],
                       'Disk4': ['#FF6666', '#FF3300']}
    edgecolor_table = {'All': ['#005F77', '#005F77'],
                       'Barrel': ['#007700', '#007700'],
                       'Disk1,2,3': ['#000775', '#000775'],
                       'Disk4': ['#CC0000', '#CC0000']}
    hatches = ['///', None]
    is_region = np.vectorize(lambda item: item.startswith(region))
    facecolors = facecolor_table['All']
    edgecolors = edgecolor_table['All']
    
    if region == 'All':
        facecolors = facecolor_table[region]
        edgecolors = edgecolor_table[region]
        is_region = np.vectorize(lambda item: type(item) is str)
    elif region == 'Barrel':
        facecolors = facecolor_table[region]
        edgecolors = edgecolor_table[region]
        is_region = np.vectorize(lambda item: item.startswith('W'))
    elif region == 'Endcap':
        facecolors = facecolor_table['Disk1,2,3']
        edgecolors = edgecolor_table['Disk1,2,3']
        is_region = np.vectorize(lambda item: item.startswith('RE'))
    elif region == 'Disk1,2,3':
        facecolors = facecolor_table[region]
        edgecolors = edgecolor_table[region]
        is_region = np.vectorize(lambda item: item.startswith(('RE+1', 'RE+2', 'RE+3', 'RE-1', 'RE-2', 'RE-3')))
    elif region == 'Disk4':
        facecolors = facecolor_table[region]
        edgecolors = edgecolor_table[region]
        is_region = np.vectorize(lambda item: item.startswith(('RE+4', 'RE-4')))
    elif region.startswith('W'):
        facecolors = facecolor_table['Barrel']
        edgecolors = edgecolor_table['Barrel']
    elif region.startswith(('RE+1', 'RE+2', 'RE+3', 'RE-1', 'RE-2', 'RE-3')):
        facecolors = facecolor_table['Disk1,2,3']
        edgecolors = edgecolor_table['Disk1,2,3']
    elif region.startswith(('RE+4', 'RE-4')):
        facecolors = facecolor_table['Disk4']
        edgecolors = edgecolor_table['Disk4']
    
    return {
        'is_region': is_region,
        'facecolors': facecolors,
        'edgecolors': edgecolors,
        'hatches': hatches
    }

def hist_eff_by_roll(total_passed_rolls_1,
                     total_passed_rolls_2,
                     region,
                     output_path):
    total_1, passed_1, roll_name_1 = total_passed_rolls_1[0], total_passed_rolls_1[1], total_passed_rolls_1[2]
    total_2, passed_2, roll_name_2 = total_passed_rolls_2[0], total_passed_rolls_2[1], total_passed_rolls_2[2]
    
    region_params = get_region_params(region)
    is_iRPC = np.vectorize(lambda roll: roll in {"RE+4_R1_CH15_A", "RE+4_R1_CH16_A", "RE+3_R1_CH15_A", "RE+3_R1_CH16_A"})

    total_1 = total_1[region_params['is_region'](roll_name_1) & ~is_iRPC(roll_name_1)]
    passed_1 = passed_1[region_params['is_region'](roll_name_1) & ~is_iRPC(roll_name_1)]
    
    total_2 = total_2[region_params['is_region'](roll_name_2) & ~is_iRPC(roll_name_2)]
    passed_2 = passed_2[region_params['is_region'](roll_name_2) & ~is_iRPC(roll_name_2)]
    
    eff_1 = np.divide(passed_1, total_1,
                      out = np.zeros_like(total_1),
                      where = (total_1 > 0)) * 100

    eff_2 = np.divide(passed_2, total_2,
                      out = np.zeros_like(total_2),
                      where = (total_2 > 0)) * 100

    eff_1 = eff_1[total_1 != 0]
    eff_2 = eff_2[total_2 != 0]
    
    n_total_1 = len(total_1)
    n_eff_under_70_1 = len(eff_1[eff_1 <= 70])
    n_excluded_1 = len(total_1[total_1 == 0])

    n_total_2 = len(total_2)
    n_eff_under_70_2 = len(eff_2[eff_2 <= 70])
    n_excluded_2 = len(total_2[total_2 == 0])

    fig, ax = init_figure(
        figsize = (12, 8),
        fontsize = 20,
        com = r'$\sqrt{s} = 13.6$',
        label1 = 'Work in Progress',
        mid_label = f'RPC {region}',
        label2 = f'2022, 2023',
        loc = 0,
        xlabel = 'Efficiency [%]',
        ylabel = 'Number of Rolls',
        xlim = (70, 100),
        ylim = None,
        xticks = None,
        yticks = None,
        log_scale = False,
        #lumi=62.6,
    )

    count_1, bins_1, patch_1 = ax.hist(eff_1[eff_1 > 0], 
                                       bins = 200, 
                                       range = (0, 100),
                                       facecolor = region_params['facecolors'][0],
                                       edgecolor = region_params['edgecolors'][0],
                                       hatch = region_params['hatches'][0],
                                       alpha = 0.5,
                                       align = 'mid',
                                       density = False,
                                       linewidth = 1.6,
                                       histtype = 'stepfilled')

    count_2, bins_2, patch_2 = ax.hist(eff_2[eff_2 > 0], 
                                       bins = 200,
                                       range = (0, 100),
                                       facecolor = region_params['facecolors'][1],
                                       edgecolor = region_params['edgecolors'][1],
                                       hatch = region_params['hatches'][1],
                                       alpha = 0.5,
                                       align = 'mid',
                                       density = False,
                                       linewidth = 1.6,
                                       histtype = 'stepfilled')
    
    extra = Rectangle((0, 0), 0.1, 0.1, fc='w', fill=False, edgecolor='none', linewidth=0)
    header_row = ['',
                  '',
                  r'$<eff>$',
                  r'$<eff>_{eff>70\ \%}$',
                  r'$N_{total}$',
                  r'$N_{eff<70\ \%}$',
                  r'$N_{excluded}$',]

    data_1_row = ['', 
                  r'$Data_{run<369374}$', 
                  f' {np.mean(eff_1):.1f} %', 
                  f' {np.mean(eff_1[eff_1 > 70]):.1f} %',
                  f'{n_total_1}',
                  f'{n_eff_under_70_1} ({n_eff_under_70_1/n_total_1*100:.1f} %)',
                  f'{n_excluded_1} ({n_excluded_1/n_total_1*100:.1f} %)',]

    data_2_row = ['', 
                  r'$Data_{run\geq369374}$', 
                  f' {np.mean(eff_2):.1f} %', 
                  f' {np.mean(eff_2[eff_2 > 70]):.1f} %',
                  f'{n_total_2}',
                  f'{n_eff_under_70_2} ({n_eff_under_70_2/n_total_2*100:.1f} %)',
                  f'{n_excluded_2} ({n_excluded_2/n_total_2*100:.1f} %)',]

    legend_handles, legend_values = [], []
    for idx in range(len(header_row)):
        if idx == 0:
            legend_handles += [extra, patch_1[0], patch_2[0]]
        else:
            legend_handles += [extra, extra, extra]
        legend_values += [header_row[idx], data_1_row[idx], data_2_row[idx]]

    ax.legend(legend_handles, legend_values,
              ncol = len(header_row), columnspacing = 0.0,
              handletextpad = -0.6, handlelength = 1.5,
              alignment = 'center', loc = 'upper left', fontsize = 18)
    output_path = Path(output_path)
    if not output_path.parent.exists():
        output_path.parent.mkdir(parents=True)
    fig.savefig(output_path)
    plt.close(fig)
    return [eff_1, eff_2]



In [3]:
total_by_roll_run = uproot.open(f'/users/hep/eigen1907/store/TnP-NanoAOD/analysis/with_blacklist_roll_run/Run3.root:total_by_roll_run').to_hist()
passed_by_roll_run = uproot.open(f'/users/hep/eigen1907/store/TnP-NanoAOD/analysis/with_blacklist_roll_run/Run3.root:passed_by_roll_run').to_hist()
roll_names = np.array(total_by_roll_run.axes[0])
runs = np.array(total_by_roll_run.axes[1])


total = total_by_roll_run.values()
passed = passed_by_roll_run.values()

roll_names = np.array(total_by_roll_run.axes[0])
runs = np.array(total_by_roll_run.axes[1])

re4issue_runs_mask = (runs >= 369374)

total_1 = np.sum(total[:, ~re4issue_runs_mask], axis=1)
total_2 = np.sum(total[:, re4issue_runs_mask], axis=1)

passed_1 = np.sum(passed[:, ~re4issue_runs_mask], axis=1)
passed_2 = np.sum(passed[:, re4issue_runs_mask], axis=1)

regions = ['All', 'Endcap', 'Disk1,2,3', 'Disk4']
for region in regions:
    hist_eff_by_roll([total_1, passed_1, roll_names],
                     [total_2, passed_2, roll_names],
                     region,
                     f'/users/hep/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/plotting/hist_eff_by_roll/resolve_re4_issue/{region}.png')
